<a href="https://colab.research.google.com/github/Nagmashaik123/MedConcepts/blob/main/TweetSimilarityCheckSample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [82]:
import pandas as pd
df=pd.read_csv('TweetSimilarityCheckSample.csv')
df

,CUID,Concept,Definition,TweetText,HashTags,CleanedTweet
0,C0863129,AGEP,Acute generalized exanthematous pustulosis (AG...,['#ArchivoGeneralEstadoPuebla #AGEP #Puebla #A...,"['ArchivoGeneralEstadoPuebla, AGEP, Puebla, Ar...",['archivogeneralestadopuebla age preble archiv...
1,C1302752,Abrasion,\nan abraded area of the skin or mucous membrane,"[""Unfortunately, there's no evidence to sugges...","['banana, whitenteeth, dentalhealth, smile, ab...",['unfortunately there no evidence to suggest t...
2,C0000833,Abscess,\nAccumulation of purulent material in tissues...,['Yank that impacted tooth before it becomes a...,"['abscess, Traitor', 'Abscess, SewerRunsThroug...",['bank that impacted tooth before it becomes a...
3,C0234238,Ache,A dull persistent (usually moderately intense)...,['#strength the #aerialstraps way! I #ache tod...,"['strength, aerialstraps, ache, calesthenics, ...",['strength the aerialstraps way i ache today c...
4,C0001122,Acidosis,pathologic condition resulting from accumulati...,"[""Gain new insights in tumor microenvironment ...","['invivoimaging, FLI, fluorescenceimaging, tum...",['gain new insight in tumor microenvironment c...


In [ ]:
!pip install -U sentence-transformers

In [ ]:
#!pip install -U sentence-transformers
import csv
#download a pretrained model.
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Example in semantic search
from sentence_transformers import SentenceTransformer, util
import torch
file = open('TweetSimilarityCheckScore.csv', 'w')
# create the csv writer
writer = csv.writer(file)
# write a row to the csv file
writer.writerow(['CUID', 'Concept','Definition','CleanedTweet','Score'])

embedder = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences (Tweets collected)
count=0
scoreList=[]
for ind in df.index:
  eachTweetList = df["CleanedTweet"][ind].replace("'", "")
  eachTweetList = eachTweetList[1:-1].split(",")
  corpus_embeddings = embedder.encode(eachTweetList, convert_to_tensor=True)
  # Query sentences (Definitions)
  query = df['Definition'][ind]
  # Find the closest sentences of the corpus for each query sentence based on cosine similarity
  top_k = len(eachTweetList)   # it depends on us
  #for query in queries:
  query_embedding = embedder.encode(query, convert_to_tensor=True)
  cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
  cos_scores = cos_scores.cpu()
  #We use torch.topk to find the highest 5 scores
  top_results = torch.topk(cos_scores, k=top_k)

  print("\n\n======================\n\n")
  #print("Query:", query)
  print("\nTop most similar sentences in corpus:")
  scoreDict={}
  for score, idx in zip(top_results[0], top_results[1]):
    print(eachTweetList[idx], "(Score: %.4f)" % (score))
    scoreDict={"CUID":df["CUID"][ind], "Concept": df["Concept"][ind], "Definition":df['Definition'][ind],"TweetText":df['TweetText'][ind],"HashTags":df['HashTags'][ind],"CleanTweet":eachTweetList[idx],"Score":"%.4f" % score}
    scoreList.append(scoreDict)


data = pd.DataFrame(scoreList)    

In [85]:
data.to_csv('output.csv')